# **Práctica 2**
* **Alumno 1**: Bolinches Segovia, Jorge
* **Alumno 2**: Cerezo Pomykol, Jan

In [13]:
import numpy as np
import pandas as pd
import os.path
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, fetch_openml

# Iris
dataset = load_iris()
X_iris = dataset.data
y_iris = dataset.target

# Wine
dataset = load_wine()
X_wine = dataset.data
y_wine = dataset.target

# Cancer
dataset = load_breast_cancer()
X_cancer = dataset.data
y_cancer = dataset.target


# Isolet
# Si existe la base de datos, cargo las variables
if os.path.exists("isolet_X.pickle"):
    X = pd.read_pickle('isolet_X.pickle')
    y = pd.read_pickle('isolet_y.pickle')
else:
    # Cargamos desde internet ( https://www.openml.org ) y la guardamos en el directorio local
    X, y = fetch_openml('isolet', version=1, return_X_y=True, cache=False)
    # Guardamos los datos para no volver a descargarlos
    X.to_pickle("isolet_X.pickle")
    y.to_pickle("isolet_y.pickle")

X_isolet = np.array(X)
y_isolet = pd.factorize(y)[0]

# MNIST
# Si existe la base de datos, cargo las variables
if os.path.exists("mnist_X.pickle"):
    X = pd.read_pickle('mnist_X.pickle')
    y = pd.read_pickle('mnist_y.pickle')
else:
    # Cargamos desde internet ( https://www.openml.org ) y la guardamos en el directorio local
    X, y = fetch_openml('mnist_784', version=1, return_X_y=True, cache=False)
    # Guardamos los datos para no volver a descargarlos
    X.to_pickle("mnist_X.pickle")
    y.to_pickle("mnist_y.pickle")

X_mnist = np.array(X)
y_mnist = pd.factorize(y)[0]

datasets = {"iris": (X_iris, y_iris),
            "wine": (X_wine, y_wine),
            "cancer": (X_cancer, y_cancer),
            "isolet": (X_isolet, y_isolet),
            "mnist": (X_mnist, y_mnist)}

In [16]:
for i in datasets:
    print(i, ": ", datasets[i][0].shape, sep='')

iris: (150, 4)
wine: (178, 13)
cancer: (569, 30)
isolet: (7797, 617)
mnist: (70000, 784)


## Entrenamiento y evaluación del perceptrón.

In [17]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import cross_val_score

In [18]:
clf = Perceptron(tol=1e-3, random_state=0)

for i in datasets:
    scores = cross_val_score(clf, datasets[i][0], datasets[i][1], cv=10)
    print(i, ": ", "%.4f" % np.mean(scores), ", std: ", "%.4f" % np.std(scores), sep='')




iris: 0.7067, std: 0.0611
wine: 0.6134, std: 0.1230
cancer: 0.8964, std: 0.0345
isolet: 0.9433, std: 0.0181
mnist: 0.8671, std: 0.0123


## Ensemble del clasificador.